In [262]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import imageio


In [263]:

def SOR_iteration(c, w, N, epsilon = 1e-5, max_iter=10000):
    c_next = c.copy()
    results = {}

    for k in range(max_iter): 
        for j in range(1, N):  # Skip boundaries in x-direction
            for i in range(1, N):  # Skip boundaries in y-direction
                c_next[i, j] = (
                    w * 0.25
                    * (c[i + 1, j] + c_next[i - 1, j] + c[i, j + 1] + c_next[i, j - 1])
                    + (1 - w) * c[i,j]
                )

        # Apply periodic boundary conditions in the x-direction
        for j in range (1, N):
            c_next[N,j] = (
                    w * 0.25
                    * (c[1, j] + c_next[N - 1, j] + c[N, j + 1] + c_next[N, j - 1])
                    + (1 - w) * c[N, j]
                )
            
        c_next[0, 1:N] = c_next[N, 1:N]

        # Apply fixed boundary conditions
        c_next[:, N] = 1.0  # Top boundary
        c_next[:, 0] = 0.0  # Bottom boundary

        delta = np.max(np.abs(c_next-c))

        results[k] = {"c": c_next.copy()[0], "delta": delta}

        if delta < epsilon:
            #print("SOR: ", k)
            return c_next
        else:
            c[:] = c_next[:]


In [264]:
def get_neighbours(i,j): 
    return [[i, j-1],[i, j+1],[i+1, j],[i-1, j]]

def get_growth_candidates(grid, N): 
    candidates = []
    for i in range(N): 
        for j in range(N): 
            if grid[i,j]:
                for ni, nj in get_neighbours(i,j):
                    if 0 < ni < N and 0 < nj < N:
                        candidates.append([ni,nj])    
    
    return candidates
    

def get_p(diffusion_grid, candidates, eta): 
    weights = np.array([diffusion_grid[i, j]**eta for i, j in candidates])
    return weights / np.sum(weights)  # Normalize probabilities



def dla(c, cluster_grid, w, N, timesteps , eta, epsilon = 1e-5, max_iter=10000): 
    growth_evolution = {}
    for timestep in range(timesteps): 
        
        diffusion_grid = SOR_iteration(c, w, N)
        
        candidates = get_growth_candidates(cluster_grid, N)

        if not candidates:
            print(f"No grotwh possibilities for the cluster left (after {timestep} timesteps)")
            break 
        p = get_p(diffusion_grid, candidates, eta)
        
        c_i, c_j = candidates[np.random.choice(len(candidates), p = p)]
        print(c_i, c_j)
        cluster_grid[c_i, c_j] = 1 
        growth_evolution[timestep] = cluster_grid

    return growth_evolution
        


In [265]:
## Parameters ## 
N = 10 
eta = 1.0 
steps = 100 


In [266]:
## initialize ## 

c = np.zeros((N+1,N+1))
#place seed at bottom centre 
cluster_grid = np.zeros((N+1,N+1)) #diffusion grid 
cluster_grid[N-1, N//2] = 1 



In [267]:
growth_evolution = dla(c, cluster_grid, 1.8, N, 50, eta)

8 5
9 5
8 6
8 5
8 4
9 4
8 6
9 6
7 4
7 5
7 6
7 5
8 6
6 4
9 6
9 7
8 5
9 7
9 8
9 6
7 4
9 3
7 5
6 5
7 4
9 4
7 4
8 6
8 7
8 5
9 5
9 6
8 5
8 5
9 9
9 7
6 5
9 8
9 8
8 6
6 5
9 5
9 6
8 9
9 7
8 4
7 4
9 5
7 7
6 5


In [268]:
def create_gif(growth_evolution, filename='cluster_growth.gif', interval=100):
    """
    Create a GIF of the cluster spreading over time.

    Parameters:
    growth_evolution (dict): Dictionary containing the cluster grid at each timestep.
    filename (str): Name of the output GIF file.
    interval (int): Time interval between frames in milliseconds.
    """
    frames = []
    for timestep in sorted(growth_evolution.keys()):
        fig, ax = plt.subplots()
        ax.imshow(growth_evolution[timestep], cmap='Greys', interpolation='nearest')
        ax.set_title(f'Timestep {timestep}')
        ax.axis('off')
        
        # Save the current frame
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(image)
        plt.close(fig)
    
    # Create the GIF
    imageio.mimsave(filename, frames, duration=interval / 1000)


create_gif(growth_evolution)

C:\Users\imaan\AppData\Local\Temp\ipykernel_19708\1351722370.py:19: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
